In [2]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map

Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [3]:
# -------------------------------------------------------
# 2  Image operator(mask, clip, select, addBands, export)
#     a.Image数据的基础操作
# -------------------------------------------------------
# 根据常量创建ee对象
image1 = ee.Image(1)
Map.addLayer(image1,{min: -1e3, max: 5e3},'image1')

In [4]:
# ee对象的合并
image2 = ee.Image(2)
image3 = ee.Image(3)
image4 = ee.Image.cat([image1, image2, image3])
print(image4.getInfo())



{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [5]:
# 向ee对象中添加波段i
image5 = image4.addBands(ee.Image(42))
print(image5.getInfo())


{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'constant_3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 42, 'max': 42}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [6]:
# ee对象修改波段名 select的用法   重点！！！！！！！！！！！
renamed = image4.select(
    [0, 1, 2], # old names
    ['band1', 'band2', 'band3']               # new names
)
print(renamed.getInfo())


{'type': 'Image', 'bands': [{'id': 'band1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'band2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'band3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [7]:
# 导入ee图层中的指定波段————select函数，select有两种传递的参数方式
# 第一种直接传递波段索引值或波段名称或其名称的正则表达式组成的非列表序列，如下：
loadedImage = ee.Image('JAXA/ALOS/AW3D30/V2_2').select('.*?DSM', 'AVE_STK', 2)
print(loadedImage.bandNames().getInfo())
Map.addLayer(loadedImage, {}, 'loadedImage')

['AVE_DSM', 'AVE_STK', 'AVE_MSK']


In [8]:
# 第二种传递两个列表，第一个列表用于波段筛选，第二个列表用于指定波段的新名称
# 两个列表的元素个数必须一致，如下：
loadedImage_All = ee.Image('JAXA/ALOS/AW3D30/V2_2').select([0, 1], ['A', 'B'])
print(loadedImage_All.getInfo())
Map.addLayer(loadedImage_All, {}, 'All')
Map.addLayer(loadedImage_All.select(0), {}, 'A')
Map.addLayer(loadedImage_All.select('B'), {}, 'B')

{'type': 'Image', 'bands': [{'id': 'A', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296000, 597600], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180, 0, -0.0002777777777777778, 84]}, {'id': 'B', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [1296000, 597600], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180, 0, -0.0002777777777777778, 84]}], 'id': 'JAXA/ALOS/AW3D30/V2_2', 'version': 1641990176334383, 'properties': {'type_name': 'Image', 'keywords': ['alos', 'dem', 'elevation', 'geophysical', 'jaxa', 'topography'], 'thumb': 'https://mw1.google.com/ges/dd/images/AW3D30_thumb.png', 'description': '<p>ALOS World 3D - 30m (AW3D30) is a global digital surface\nmodel (DSM) dataset with a horizontal resolution of approximately\n30 meters (1 arcsec mesh). The dataset is based on the DSM dataset\n(5-meter mesh version) of the <a href="https://www.aw3d.jp

In [9]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
# 影像数据的波段名字
bandNames = image.bandNames().getInfo()
print(f'band :{bandNames}')
# 波段B3的投影信息
b1proj = image.select('B3').projection().getInfo()
print(f'projection :{b1proj}')
# 波段B4的像元大小
b1scale = image.select('B4').projection().nominalScale().getInfo()
print(f'scale :{b1scale}')
# 影像数据的属性名称
properties = image.propertyNames().getInfo()
print(f'properties: {properties}')
# 影像数据的云量
cloudiness = image.get('CLOUD_COVER').getInfo()
print(f'CLOUD_COVER: {cloudiness}')
# 影像数据的时间（格式已转换）
date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss').getInfo()
print(f'date :{date}')

band :['B3', 'B4', 'B5']
projection :{'type': 'Projection', 'crs': 'EPSG:32610', 'transform': [30, 0, 460785, 0, -30, 4264215]}
scale :30
properties: ['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'CPF_NAME', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'google:registration_count', 'PANCHROMATIC_SAMPLES', 'PANCHROMAT

In [10]:
# 可通过image.propertyNames()获取属性名字
props = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').propertyNames().getInfo()
print(props)

['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'CPF_NAME', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'google:registration_count', 'PANCHROMATIC_SAMPLES', 'PANCHROMATIC_LINES', 'GEOMETRIC_RMSE_MODEL_Y', 'REFLECTIVE_LINES', 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE', 'GEOMETRIC_RMSE_MODEL_X', 'system:asset_size', 'system:

In [11]:
# 可通过get().getInfo()获取属性名字中的常量
ID = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').get('system:id').getInfo()
print(ID)

LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318


In [12]:
# // Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')

# // Create NDVI and NDWI spectral indices.（计算两个波段之间的归一化差值）
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])
print(f'ndvi: {ndvi.getInfo()}\n ndwi: {ndwi.getInfo()}')

# // Create a binary layer using logical operations.
# 获取ndvi小于0.2和ndwi小于0的数据
bare = ndvi.lt(0.2).And(ndwi.lt(0))
print(f'bare :{bare.getInfo()}')

# // Mask and display the binary layer.
Map.setCenter(-122.3578, 37.7726, 12)
# 图层转化
Map.setOptions('SATELLITE')
Map.addLayer(bare.updateMask(bare), {}, 'bare')

ndvi: {'type': 'Image', 'bands': [{'id': 'nd', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-121.3637119499993, 36.41016684133052], [-121.35905784815819, 36.42528989660049], [-121.2315833015866, 36.840374852891664], [-121.09978718573184, 37.26438246506325], [-121.00571062336425, 37.564795515259384], [-120.98453376062118, 37.632161601008896], [-120.95100979452299, 37.73864548098522], [-120.90277241165228, 37.89149086576169], [-120.8836409072059, 37.951976016520376], [-120.85713152433351, 38.03584247073611], [-120.82804345546616, 38.12789513604401], [-122.38148159443172, 38.42337450676813], [-122.9500220192271, 38.525813632077686], [-122.95103687833704, 38.52422133103557], [-122.9569591344694, 38.504384836247866], [-123.43853932998316, 36.805122381748035], [-123.18722447462653, 36

In [14]:
# ------------------------------------------------------------------------------------------
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# ------------------------------------------------------------------------------------------
# // Load a 2012 nightlights image.
# 选取2012年数据集中stable_lights数据（光污染）
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# // Define arbitrary thresholds on the 6-bit stable lights band.
# 将大于30的数据加上大于55的数据再加上大于62的数据（参数的选取和数据有关）
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# // Display the thresholded image as three distinct zones near Paris.
# 设置数据颜色
palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {min: 0, max: 3, 'palette': palette}, 'development zones')

In [15]:
# -------------------------------------------------------
# 2  Image operator(export)
#     b.export an Image to Google drive
# -------------------------------------------------------
# max: 3784216,672,400 pixels
# geemap.ee_export_image(image, filename=file, scale=1000, region=roi,file_per_band=True)
# 将数据导入到云盘中
geemap.ee_export_image_to_drive(image, description="LC08_044034_20140318", folder="export", scale=30)

In [16]:
# -------------------------------------------------------
# 2  Image operator(export)
#     b.export an Image to local directory
# -------------------------------------------------------
# max: 3784216,672,400 pixels
# 将数据导入到本地中
f_img = r"C:\Users\xiong\Downloads\LC08_044034_20140318.tif"
geemap.ee_export_image(image, filename=f_img, scale=300, region=roi, file_per_band=True)
# geemap.ee_export_image_to_drive(image, description="LC08_044034_20140318", folder="export", scale=30)

NameError: name 'roi' is not defined

In [36]:
# -------------------------------------------------------
# 2  Image operator(export)
#     b.export an Image to local directory
# -------------------------------------------------------
# from Gee import ee_export

image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
print(image)
# f_img = r"C:\Users\YeHui\Downloads\LC08_044034_20140318b"   # do not use name of extension for image
# roi = image.geometry()

# ee_export(region=roi, image=image, outfile=f_img, scale=30, sep=0.25, num_workers=None)
# ------------------------------------------------------------------------------------------
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# ------------------------------------------------------------------------------------------
# def ee_export(region: ee.Geometry, image: ee.Image, outfile: str, scale: int,
#                    crs='epsg:4326', sep=0.2):
#     """
#     Args:
#         geo: ee.Geometry, 需要下载的区域矢量几何
#         image: ee.Image, 需要下载的影像
#         outfile: str, 输出文件路径和名称，不需要文件后缀，下载的影响默认后缀为tif
#         scale: int, 下载时的像元大小
#         crs: str, 下载影像的投影，默认为 'epsg:4326' wgs1984投影
#         sep: float, 单波段10m分辨率像元的影像裁剪大小(单位：经纬度)，默认为0.2
# ------------------------------------------------------------------------------------------
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# ------------------------------------------------------------------------------------------

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.select",
    "arguments": {
      "bandSelectors": {
        "constantValue": [
          "B3",
          "B4",
          "B5"
        ]
      },
      "input": {
        "functionInvocationValue": {
          "functionName": "Image.load",
          "arguments": {
            "id": {
              "constantValue": "LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318"
            }
          }
        }
      }
    }
  }
})


In [ ]:
# -------------------------------------------------------
# 2  Image operator(export)
#     b.export a ImageCollection to local directory
# -------------------------------------------------------